In [108]:
import yaml
import json
from openai import OpenAI
import sqlite3
import pandas as pd
import time
import sqlglot

import pprint
pp = pprint.PrettyPrinter(indent=4)

In [114]:
with open('api_keys.yaml', 'r') as f:
    api_keys = yaml.safe_load(f)

OPENAI_API_KEY = api_keys['openai-key']
client = OpenAI(api_key=OPENAI_API_KEY)

In [110]:
db_name = "lldm.db"
try: 
    db = sqlite3.connect(db_name) 
    print('Connected to DB')
except: 
    print("DB connection failed")

Connected to DB


In [111]:
# run query and return pandas dataframe
def run_query(db, query):
    try:
        df = pd.read_sql_query(query, db)
        return df
    # not a select statement
    except TypeError:
        cursor = db.cursor()
        cursor.execute(query)
        return    

## Assistant initialize

In [119]:
# get_info_func_desc = '''
# Generate a SQL query to help extract the information that the user is asking for with regards to the items in their character's inventory. Use the following view to help generate the query:

# CREATE TABLE CHARACTER_INVENTORY_DETAILS (
#     Campaign_ID int,
#     Character_ID int, 
#     Item_ID int, 
#     Quantity int,
#     Weapon_Name text, 
#     Weapon_Description text
# )

# Ignore the ID columns when generating the query.
# Only return the SQL query without any preamble or post text, as well as without any quotes. Do not add a semicolon at the end of the query
# Only create SELECT queries and do not create any queries that will modify the table in any way.
# '''

get_info_func_desc = '''
Generate a SQL query to help extract what the user is asking regarding the items in their character's inventory, such as taking out or utilizing an item in posession. 
Use the following view to help generate the query, with the description of each column in between asterisks (*):

CREATE TABLE IF NOT EXISTS CHARACTER_INVENTORY_DETAILS (
    Weapon_Name text, 
    Weapon_Description text ,
    Total_Quantity int    
);

CREATE TABLE IF NOT EXISTS CHARACTER_INVENTORY_HISTORY_DETAILS (
    Weapon_Name text, 
    Weapon_Description text,
    Quantity int *A positive number represents obtained weapons, while a negative number means discarded*,
    Modify_Time datetime
);

Make sure to only query columns that exist from each table. Do not switch column-table identities.
Only return the SQL query without any preamble or post text, as well as without any quotes. Do not add a semicolon at the end of the query
Only create SELECT queries and do not create any queries that will modify the table in any way.
'''

In [120]:
# create the narrator assistant

narrator = client.beta.assistants.create(
    name="narrator",
    instructions=
        """
        You are a DnD DM. You sets the scene by describing the environment and atmosphere, brings NPCs to life through detailed character portrayals, and narrates the outcomes of player actions. They establish the game's tone, provide world-building lore, guide the overarching story while balancing player choices, and enforce game rules.

        The information of the main character is as follows: Elara Windrider, a courageous warrior with a heart of gold, is a human fighter who embodies the principles of Lawful Good. She is tall and athletic, with short brown hair, green eyes, and a determined expression. Clad in chain mail and wielding a longsword, Elara's appearance reflects her readiness for battle. Born in a small village, she was trained by her father, a retired soldier. Driven by a desire to protect the innocent and seek justice, she left home to make her mark on the world. Elara is brave and compassionate, possessing a strong sense of justice. Though she is determined and reliable, her stubbornness can sometimes get the best of her.

        The plot summary is as follows:

            The Dragon's Flagon (Tavern)
        Description: The Dragon's Flagon is a lively tavern with a warm, welcoming atmosphere. The walls are adorned with trophies from past adventurers, and a large fireplace dominates one side of the room.
        
            Whispering Woods (Wilderness)
        Description: Whispering Woods is a foreboding forest with a canopy so thick it blocks out most of the sunlight. The air is filled with the sounds of unseen creatures, and the ground is covered with a thick layer of leaves.
        
        Description: Blackstone Keep is a crumbling fortress with tall, dark towers and walls covered in ivy. Inside, it is dark and cold, with the air thick with the smell of decay.

        After receiving user response, you generate a narrative that moves the plot forward while maintaining a realistic continuity of events.
        """,

    tools=[
        # {"type": "file_search"},
        {
            "type": "function",
            "function": {
                "name": "get_obtained_item",
                "description": "Extract the item that the user has obtained in some manner (such as picked up, purchased, etc.). This does not include utilizing an item that the user currently might have in their inventory.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "item_name": {
                            "type": "string",
                            "description": "The name of the obtained item.",
                        },
                        "quantity": {
                            "type": "integer",
                            "description": "The number of said items obtained. If a number is not specified, try and infer based on the surrounding context."
                        }
                    },
                    "required": ["item_name", "quantity"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_discarded_item",
                "description": "Extract the item that the user has discarded in some manner (such as thrown away, consumed, broken, etc.)",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "item_name": {
                            "type": "string",
                            "description": "The name of the discarded item.",
                        },
                        "quantity": {
                            "type": "integer",
                            "description": "The number of said items discarded. If a number is not specified, try and infer based on the surrounding context."
                        }
                    },
                    "required": ["item_name", "quantity"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_item_info",
                "description": get_info_func_desc,
                "parameters": {
                    "type": "object",
                    "properties": {
                        "sql_query": {
                            "type": "string",
                            "description": "The generated SQL query that would extract the information related to the character inventory requested by the user",
                        }
                    },
                    "required": ["sql_query"],
                },
            },
        }
    ],
    model="gpt-3.5-turbo",
)


## Defining functions

In [121]:
# defining custom exceptions to be used with error handling
class ItemNotFoundException(Exception):
    pass

class ItemNotPossessedException(Exception):
    pass

In [135]:
# check if item queried is in INVENTORY table
def validate_item(item_name):
    query = f'''
    SELECT * FROM WORLD_ITEMS WHERE UPPER(Weapon_Name) LIKE "{item_name.upper()}%" 
    '''
    tmp = run_query(db, query)
    if not tmp.empty:
        return tmp['Item_ID'].iloc[0]
    return None

# update table if item validated, otherwise error message
# for now, use temporary campaign and character id
def get_obtained_item(item_name, quantity, campaign_id=0, character_id=0):
    item_id = validate_item(item_name)
    print(item_id, quantity)
    if item_id is not None:
        # TODO: error handling
        cursor = db.cursor()

        # overall character inventory tracker update
        query = f'''
        UPDATE CHARACTER_INVENTORY SET Total_Quantity = Total_Quantity + {quantity}
        WHERE Item_ID = {item_id} AND Campaign_ID = {campaign_id} AND Character_ID = {character_id}
        '''
        cursor.execute(query)

        if cursor.rowcount == 0:
            query = f'''
            INSERT INTO CHARACTER_INVENTORY (Campaign_ID, Character_ID, Item_ID, Total_Quantity) VALUES ({campaign_id}, {character_id}, {item_id}, {quantity})
            '''
            cursor.execute(query)

        # inventory history update
        query = f'''
        INSERT INTO CHARACTER_INVENTORY_HISTORY (Campaign_ID, Character_ID, Item_ID, Quantity) VALUES ({campaign_id}, {character_id}, {item_id}, {quantity})
        '''
        cursor.execute(query)

        return json.dumps({'message':'The item(s) were successfully obtained. Please continue the story.'})
    else:
        return json.dumps({'message':'Item does not exist. Please prompt user to specify further or provide another action.'})


# check if item queried is in INVENTORY table and character has more than discard amount
# might need a third error condition if item exists, user has item, but has less than discard amount
def validate_item_discard(item_name, quantity, campaign_id, character_id):
    query = f'''
    SELECT * FROM WORLD_ITEMS WHERE UPPER(Weapon_Name) LIKE "{item_name.upper()}%" 
    '''
    tmp = run_query(db, query)
    if not tmp.empty: # item exists in world
        item_id = tmp['Item_ID'].iloc[0]
        # validate if character has item
        query = f'''
        SELECT * FROM CHARACTER_INVENTORY 
        WHERE Campaign_ID = {campaign_id} AND Character_ID = {character_id} AND Item_ID = {item_id} AND Total_Quantity >= {quantity}
        '''
        # there should theoretically only be one row of the item for each character with the quantity as different values
        # need to validate and put checks in place
        tmp = run_query(db, query)
        if not tmp.empty:
            return item_id
        raise ItemNotPossessedException("Character does not have the item in their inventory.")
    raise ItemNotFoundException("Item does not exist in this campaign.")

# update table if item validated, otherwise error message
# for now, use temporary campaign and character id
# first update with item subtraction, then remove all entries with <=0 values
def get_discarded_item(item_name, quantity, campaign_id=0, character_id=0):
    try:
        item_id = validate_item_discard(item_name, quantity, campaign_id, character_id)
        print(item_id, quantity)
        cursor = db.cursor()

        # overall inventory tracker update
        query = f'''
        UPDATE CHARACTER_INVENTORY SET Total_Quantity = Total_Quantity - {quantity}
        WHERE Item_ID = {item_id} AND Campaign_ID = {campaign_id} AND Character_ID = {character_id};
        '''
        cursor.execute(query)

        # housekeeping query, remove rows with invalid quantites (<= 0 items)
        query = 'DELETE FROM CHARACTER_INVENTORY WHERE Total_Quantity <= 0'
        cursor.execute(query)

        # inventory history tracker update
        # use negative quantity value to indicate 
        query = f'''
        INSERT INTO CHARACTER_INVENTORY_HISTORY (Campaign_ID, Character_ID, Item_ID, Quantity) VALUES ({campaign_id}, {character_id}, {item_id}, {-quantity})
        '''
        cursor.execute(query)

        return json.dumps({'message':'The item(s) were successfully discarded. Please continue the story.'})
    except ItemNotPossessedException as e:
        return json.dumps({'message':"Item is not in character's possession. Please prompt user to specify further or provide another action."})
    except ItemNotFoundException as e:
        return json.dumps({'message':"Item does not exist. Please prompt user to specify further or provide another action."})


# get info for item queried
# used for when questions on the item itself are asked, but no changes are necessary
# def get_item_info(item_name, campaign_id=0, character_id=0):
#     try:
#         # item validation is essentially the same as for discard, but do not care about quantity
#         item_id = validate_item_discard(item_name, 1, campaign_id, character_id)
#         print(item_id)
#         query = f'''
#         SELECT Weapon_Name, Weapon_Description FROM WORLD_ITEMS
#         WHERE Item_ID = {item_id}
#         '''
#         df_item = run_query(db, query)
#         item_name = df_item['Weapon_Name'].iloc[0]
#         item_desc = df_item['Weapon_Description'].iloc[0]
#         return json.dumps({'message':f'The item mentioned is {item_name}, with a provided description of "{item_desc}". Please use this information when continuing the story.'})
#     except ItemNotPossessedException as e:
#         return json.dumps({'message':"Item is not in character's possession. Please prompt user to specify further or provide another action."})
#     except ItemNotFoundException as e:
#         return json.dumps({'message':"Item does not exist. Please prompt user to specify further or provide another action."})
    

def get_item_info(sql_query, campaign_id=0, character_id=0):
    run_query(db, 'PRAGMA QUERY_ONLY = ON;')
    try:
        
        # if 'WHERE' not in sql_query:
        #     sql_query += ' WHERE '
        # else:
        #     sql_query += ' AND '
        # sql_query += f'Campaign_ID={campaign_id} AND Character_ID={character_id}'

        # at some point in life, will figure out how to remove existing ID matching clauses before adding these
        # but for now this will do
        where = sqlglot.condition(f'Campaign_ID={campaign_id}').and_(f'Character_ID={character_id}')
        sql_query = sqlglot.parse_one(sql_query).where(where).sql()
        print(sql_query)

        df_result = run_query(db, sql_query)
        result = json.dumps(df_result.to_dict())
        print(result)

        run_query(db, 'PRAGMA QUERY_ONLY = OFF;')
        return json.dumps({'message':f"The result of the user's request in JSON format is {result}. Please use this to answer the user's question or honor the user's request."})
    except Exception as e:
        run_query(db, 'PRAGMA QUERY_ONLY = OFF;')
        return json.dumps({'message':"Something went wrong, please prompt the user for another action"})

## Narrator chat declaration

In [136]:
# create a thread for the narrator, the model will use the messages stored in it as the context

thread_narrator = client.beta.threads.create()

In [137]:
def narrator_chat(content):
    message = client.beta.threads.messages.create(
        thread_id=thread_narrator.id,
        role="assistant",
        content=f"""
        {content}
        """,
    )

    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread_narrator.id,
        assistant_id=narrator.id,
    )

    counter = 0
    while run.status != 'completed':
        # print(run.status)
        if run.status == 'requires_action':
            print('Function calling...')
            required_actions = run.required_action.submit_tool_outputs.model_dump()
            tool_outputs = []
            print(required_actions["tool_calls"])
            for action in required_actions["tool_calls"]:
                func_name = action['function']['name']
                arguments = json.loads(action['function']['arguments'])
                available_functions = {
                    "get_obtained_item": get_obtained_item,
                    "get_discarded_item": get_discarded_item,
                    'get_item_info': get_item_info
                }
                try:
                    function_to_call = available_functions[func_name]
                    print(arguments)
                    # function_args = json.loads(arguments)
                    # output = function_to_call(
                    #     item_name=function_args.get("item_name"),
                    #     quantity=function_args.get("quantity"),
                    # )
                    output = function_to_call(
                        **arguments
                    )
                    output_string = json.dumps(output)
                    print(output_string)
                    tool_outputs.append({
                        "tool_call_id": action['id'],
                        "output": output_string
                    })
                except: # function not found
                    raise ValueError(f"Unknown function: {func_name}")
                
            print("Submitting outputs back to the Assistant…")
            # print(run.status)
            run = client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread_narrator.id,
                run_id=run.id,
                tool_outputs=tool_outputs
            )
            # time.sleep(5) 
        if run.status == "cancelled":
            print("Run cancelled.")
            break
        if run.status == "cancelling":
            print("Run cancelling.")
        if run.status == "failed":
            print("Run failed.")
            break
        if run.status == "expired":
            print("Run expired.")
            break
        else:            
            run = client.beta.threads.runs.retrieve(
                thread_id=thread_narrator.id,
                run_id=run.id,
            )
            if counter % 10 == 0:
                print('Waiting for assistant. Status:', run.status)
                counter += 1
                time.sleep(5)  
            
        
    
    messages_narrator = client.beta.threads.messages.list(
        thread_id=thread_narrator.id
    )
    chat_history_narrator = []
    for thread_message in messages_narrator.data:
        for content_item in thread_message.content:
            role = thread_message.role
            id = thread_message.assistant_id
            item = content_item.text.value
            chat_history_narrator.append({'content': item})

    return chat_history_narrator

    # if run.status == 'completed':
    #     messages_narrator = client.beta.threads.messages.list(
    #         thread_id=thread_narrator.id
    #     )
    #     chat_history_narrator = []
    #     for thread_message in messages_narrator.data:
    #         for content_item in thread_message.content:
    #             role = thread_message.role
    #             id = thread_message.assistant_id
    #             item = content_item.text.value
    #             chat_history_narrator.append({'content': item})

    #     return chat_history_narrator
    
    # elif run.status == 'requires_action':
    #     print('Function calling...')
    #     required_actions = run.required_action.submit_tool_outputs.model_dump()
    #     tool_outputs = []
    #     for action in required_actions["tool_calls"]:
    #         func_name = action['function']['name']
    #         arguments = json.loads(action['function']['arguments'])
    #         available_functions = {
    #             "get_obtained_item": get_obtained_item,
    #             "get_discarded_item": get_discarded_item,
    #         }
    #         try:
    #             function_to_call = available_functions[func_name]
    #             print(arguments)
    #             # function_args = json.loads(arguments)
    #             function_args = arguments
    #             output = function_to_call(
    #                 item_name=function_args.get("item_name"),
    #                 quantity=function_args.get("quantity"),
    #             )
    #             output_string = json.dumps(output)
    #             tool_outputs.append({
    #                 "tool_call_id": action['id'],
    #                 "output": output_string
    #             })
    #         except: # function not found
    #             raise ValueError(f"Unknown function: {func_name}")
            
    #     print("Submitting outputs back to the Assistant…")
    #     client.beta.threads.runs.submit_tool_outputs(
    #         thread_id=thread_narrator.id,
    #         run_id=run.id,
    #         tool_outputs=tool_outputs
    #     )
    # else:
    #     print("Waiting for the Assistant to process…")
    #     time.sleep(5)   


## Testing

In [138]:
run_query(db, 'DELETE FROM CHARACTER_INVENTORY')
run_query(db, 'DELETE FROM CHARACTER_INVENTORY_HISTORY')

### Obtain item 1

In [139]:
thread_narrator = client.beta.threads.create()

# mention a non-existing weapon to test the DM

# narrator_chat("I pick up a generic axe.")

# to make Elara drop a weapon

tmp = narrator_chat("I pick up the Shadow Lance of Storm.")

# to test the DM's memory

# narrator_content = "I take out the Forgotten Sword of Flame and point it at my enemies."

# chat_history_narrator = narrator_chat(narrator_content)
# if chat_history_narrator:
#     for message in chat_history_narrator:
#         pp.pprint(message['content'])
print('\n\nInitial query:',tmp[1]['content'].strip())
print('\n\nResponse:',tmp[0]['content'])

Function calling...
[{'id': 'call_wEpFCzZH3vQdybQGrEM9JcA2', 'function': {'arguments': '{"item_name":"Shadow Lance of Storm","quantity":1}', 'name': 'get_obtained_item'}, 'type': 'function'}]
{'item_name': 'Shadow Lance of Storm', 'quantity': 1}
97 1
"{\"message\": \"The item(s) were successfully obtained. Please continue the story.\"}"
Submitting outputs back to the Assistant…
Waiting for assistant. Status: queued


Initial query: I pick up the Shadow Lance of Storm.


Response: You feel a surge of power as you pick up the Shadow Lance of Storm. The lance emanates a faint, ominous glow, its tip crackling with electrical energy. This legendary weapon is said to harness the power of storms and shadows, making it a formidable tool in battle. As you wield the lance, you sense a connection to the elements around you, feeling a newfound strength coursing through your veins.

With the Shadow Lance of Storm in hand, you make your way through the Whispering Woods, the dense forest around you s

In [140]:
# base state of inventory
query = '''
SELECT * FROM CHARACTER_INVENTORY_DETAILS;
'''
run_query(db,query)

,Campaign_ID,Character_ID,Item_ID,Total_Quantity,Weapon_Name,Weapon_Description
0,0,0,97,1.0,Shadow Lance of Storm,The Shadow Lance of Storm is a lance known thr...


### Obtain item 2

In [141]:
thread_narrator = client.beta.threads.create()

# mention a non-existing weapon to test the DM

# narrator_chat("I pick up a generic axe.")

# to make Elara drop a weapon

tmp = narrator_chat("I pick up the Forgotten Sword of Flame.")

# to test the DM's memory

# narrator_content = "I take out the Forgotten Sword of Flame and point it at my enemies."

# chat_history_narrator = narrator_chat(narrator_content)
# if chat_history_narrator:
#     for message in chat_history_narrator:
#         pp.pprint(message['content'])
print('\n\nInitial query:',tmp[1]['content'].strip())
print('\n\nResponse:',tmp[0]['content'])

Function calling...
[{'id': 'call_pTE2sXYUke8f5cEYzv9z0wwp', 'function': {'arguments': '{"item_name":"Forgotten Sword of Flame","quantity":1}', 'name': 'get_obtained_item'}, 'type': 'function'}]
{'item_name': 'Forgotten Sword of Flame', 'quantity': 1}
98 1
"{\"message\": \"The item(s) were successfully obtained. Please continue the story.\"}"
Submitting outputs back to the Assistant…
Waiting for assistant. Status: queued


Initial query: I pick up the Forgotten Sword of Flame.


Response: As Elara Windrider reaches out and grasps the Forgotten Sword of Flame, a surge of warmth and power courses through her. The legendary weapon glows with an ethereal flame, a testament to its potent abilities. Elara feels a sense of destiny intertwined with this sword, as if it has been waiting for her touch.

The tavern around her falls silent for a moment, as patrons and adventurers alike turn to witness the unfolding of this momentous event. The flames on the sword dance and flicker, casting a radia

In [142]:
# base state of inventory
query = '''
SELECT * FROM CHARACTER_INVENTORY_HISTORY_DETAILS;
'''
run_query(db,query)

,Campaign_ID,Character_ID,Item_ID,Quantity,Modify_Time,Weapon_Name,Weapon_Description
0,0,0,97,1.0,2024-07-22 00:42:09,Shadow Lance of Storm,The Shadow Lance of Storm is a lance known thr...
1,0,0,98,1.0,2024-07-22 00:42:25,Forgotten Sword of Flame,The Forgotten Sword of Flame is a sword known ...


### Utilize item 1
This has some issues, it is recognizing this as obtain rather than get item info

In [143]:
thread_narrator = client.beta.threads.create()

# mention a non-existing weapon to test the DM

# narrator_chat("I pick up a generic axe.")

# to make Elara drop a weapon

# narrator_chat("I pick up the Shadow Lance of Storm.")

# to test the DM's memory

narrator_content = "I take out the Forgotten Sword of Flame and point it at my enemies."

chat_history_narrator = narrator_chat(narrator_content)
if chat_history_narrator:
    for message in chat_history_narrator:
        pp.pprint(message['content'])

Function calling...
[{'id': 'call_IjNGLxP8inexIkaHC0WwChh4', 'function': {'arguments': '{"item_name":"Forgotten Sword of Flame","quantity":1}', 'name': 'get_obtained_item'}, 'type': 'function'}]
{'item_name': 'Forgotten Sword of Flame', 'quantity': 1}
98 1
"{\"message\": \"The item(s) were successfully obtained. Please continue the story.\"}"
Submitting outputs back to the Assistant…
Waiting for assistant. Status: queued
('As you reach for your sheathed weapon, a glint catches your eye, drawing '
 'your attention to the hilt of the Forgotten Sword of Flame. With a swift '
 'motion, you unsheathe the sword, feeling the power of the flames within it. '
 'The blade glows a fiery red, emitting a comforting warmth that contrasts '
 'with the cold surroundings of Blackstone Keep.\n'
 '\n'
 'The flames dance along the edge of the sword, casting a bright light that '
 'cuts through the darkness of the fortress. The Sword of Flame has a history '
 'of vanquishing darkness and bringing light to 

In [144]:
# base state of inventory
query = '''
SELECT * FROM CHARACTER_INVENTORY_HISTORY_DETAILS;
'''
run_query(db,query)

,Campaign_ID,Character_ID,Item_ID,Quantity,Modify_Time,Weapon_Name,Weapon_Description
0,0,0,97,1.0,2024-07-22 00:42:09,Shadow Lance of Storm,The Shadow Lance of Storm is a lance known thr...
1,0,0,98,1.0,2024-07-22 00:42:25,Forgotten Sword of Flame,The Forgotten Sword of Flame is a sword known ...
2,0,0,98,1.0,2024-07-22 00:42:37,Forgotten Sword of Flame,The Forgotten Sword of Flame is a sword known ...


### Dropping item

In [149]:
thread_narrator = client.beta.threads.create()

# mention a non-existing weapon to test the DM

# narrator_chat("I pick up a generic axe.")

# to make Elara drop a weapon

# narrator_chat("I pick up the Shadow Lance of Storm.")

# to test the DM's memory

narrator_content = "Elara wanders in the wilderness and starts to feel that she is carrying too many items with her. She checks through her inventory, takes out Forgotten Sword of Flame, and discard it on the ground, before continuing her adventure."

chat_history_narrator = narrator_chat(narrator_content)
if chat_history_narrator:
    for message in chat_history_narrator:
        pp.pprint(message['content'])

Function calling...
[{'id': 'call_upnl5nNoTYs32gcGMMpOnN1W', 'function': {'arguments': '{"item_name":"Forgotten Sword of Flame","quantity":1}', 'name': 'get_discarded_item'}, 'type': 'function'}]
{'item_name': 'Forgotten Sword of Flame', 'quantity': 1}
98 1
"{\"message\": \"The item(s) were successfully discarded. Please continue the story.\"}"
Submitting outputs back to the Assistant…
Waiting for assistant. Status: queued
('Elara carefully removes the Forgotten Sword of Flame from her inventory and '
 'places it on the ground, deciding to part ways with it as she feels it no '
 'longer serves her purpose. With a determined expression, she continues her '
 'journey through the Whispering Woods, the dense forest whispering secrets '
 "around her as she treads onward. The air is thick with mystery, and Elara's "
 'senses are heightened as she remains vigilant, ready for whatever challenges '
 'may come her way.')
('\n'
 '        Elara wanders in the wilderness and starts to feel that she

In [150]:
# base state of inventory
query = '''
SELECT * FROM CHARACTER_INVENTORY_HISTORY_DETAILS;
'''
run_query(db,query)

,Campaign_ID,Character_ID,Item_ID,Quantity,Modify_Time,Weapon_Name,Weapon_Description
0,0,0,97,1.0,2024-07-22 00:42:09,Shadow Lance of Storm,The Shadow Lance of Storm is a lance known thr...
1,0,0,98,1.0,2024-07-22 00:42:25,Forgotten Sword of Flame,The Forgotten Sword of Flame is a sword known ...
2,0,0,98,1.0,2024-07-22 00:42:37,Forgotten Sword of Flame,The Forgotten Sword of Flame is a sword known ...
3,0,0,98,-1.0,2024-07-22 00:42:52,Forgotten Sword of Flame,The Forgotten Sword of Flame is a sword known ...
4,0,0,98,-1.0,2024-07-22 00:46:50,Forgotten Sword of Flame,The Forgotten Sword of Flame is a sword known ...


In [151]:
# base state of inventory
query = '''
SELECT * FROM CHARACTER_INVENTORY_DETAILS;
'''
run_query(db,query)

,Campaign_ID,Character_ID,Item_ID,Total_Quantity,Weapon_Name,Weapon_Description
0,0,0,97,1.0,Shadow Lance of Storm,The Shadow Lance of Storm is a lance known thr...


### Question asking 1

In [105]:
thread_narrator = client.beta.threads.create()

# mention a non-existing weapon to test the DM

# narrator_chat("I pick up a generic axe.")

# to make Elara drop a weapon

# narrator_chat("I pick up the Shadow Lance of Storm.")

# to test the DM's memory

# narrator_content = "How many weapons do I have?"

# chat_history_narrator = narrator_chat(narrator_content)
# if chat_history_narrator:
#     for message in chat_history_narrator:
#         pp.pprint(message['content'])

tmp = narrator_chat("How many weapons do I have?")

print('\n\nInitial query:',tmp[1]['content'].strip())
print('\n\nResponse:',tmp[0]['content'])

Function calling...
[{'id': 'call_5w0e46VNqYVRdIFmRSJIcHGX', 'function': {'arguments': '{"sql_query":"SELECT Weapon_Name, Weapon_Description, Total_Quantity FROM CHARACTER_INVENTORY_DETAILS"}', 'name': 'get_item_info'}, 'type': 'function'}]
{'sql_query': 'SELECT Weapon_Name, Weapon_Description, Total_Quantity FROM CHARACTER_INVENTORY_DETAILS'}
SELECT Weapon_Name, Weapon_Description, Total_Quantity FROM CHARACTER_INVENTORY_DETAILS WHERE Campaign_ID = 0 AND Character_ID = 0
{"Weapon_Name": {"0": "Shadow Lance of Storm", "1": "Forgotten Sword of Flame"}, "Weapon_Description": {"0": "The Shadow Lance of Storm is a lance known throughout the realms for its incredible power. Forged in the fires of an ancient forge, this lance is imbued with the essence of mage. It is said that those who wield it gain unmatched abilities, making them formidable in any battle.", "1": "The Forgotten Sword of Flame is a sword known throughout the realms for its incredible power. Forged in the fires of an ancient

In [106]:
run_query(db,'SELECT * FROM CHARACTER_INVENTORY_DETAILS')

,Campaign_ID,Character_ID,Item_ID,Total_Quantity,Weapon_Name,Weapon_Description
0,0,0,97,1.0,Shadow Lance of Storm,The Shadow Lance of Storm is a lance known thr...
1,0,0,98,2.0,Forgotten Sword of Flame,The Forgotten Sword of Flame is a sword known ...


### Question asking 2

In [107]:
thread_narrator = client.beta.threads.create()

# mention a non-existing weapon to test the DM

# narrator_chat("I pick up a generic axe.")

# to make Elara drop a weapon

# narrator_chat("I pick up the Shadow Lance of Storm.")

# to test the DM's memory

# narrator_content = "How many weapons do I have?"

# chat_history_narrator = narrator_chat(narrator_content)
# if chat_history_narrator:
#     for message in chat_history_narrator:
#         pp.pprint(message['content'])

tmp = narrator_chat("What was the latest weapon I obtained?")

print('\n\nInitial query:',tmp[1]['content'].strip())
print('\n\nResponse:',tmp[0]['content'])

Function calling...
[{'id': 'call_WBhCdPskCkixa1hvLMeLRcxo', 'function': {'arguments': '{"sql_query":"SELECT Weapon_Name FROM CHARACTER_INVENTORY_HISTORY_DETAILS ORDER BY Modify_Time DESC LIMIT 1"}', 'name': 'get_item_info'}, 'type': 'function'}]
{'sql_query': 'SELECT Weapon_Name FROM CHARACTER_INVENTORY_HISTORY_DETAILS ORDER BY Modify_Time DESC LIMIT 1'}
SELECT Weapon_Name FROM CHARACTER_INVENTORY_HISTORY_DETAILS WHERE Campaign_ID = 0 AND Character_ID = 0 ORDER BY Modify_Time DESC LIMIT 1
{"Weapon_Name": {"0": "Forgotten Sword of Flame"}}
"{\"message\": \"The result of the user's request in JSON format is {\\\"Weapon_Name\\\": {\\\"0\\\": \\\"Forgotten Sword of Flame\\\"}}. Please use this to answer the user's question or honor the user's request.\"}"
Submitting outputs back to the Assistant…
Waiting for assistant. Status: in_progress


Initial query: What was the latest weapon I obtained?


Response: The latest weapon you obtained was the "Forgotten Sword of Flame."
